<h1 style='text-align: center'>SQL Queries</h1>

## Getting Data From A SQL Database

### The Structure of a SQL Query

<img src='images/sql_statement.jpg'/>

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_matts
FROM student_courses
WHERE student_name = "Matt"
GROUP BY 1
HAVING COUNT(student_name) >= 3;
```

In [7]:
import sqlite3

sqlite_conn = sqlite3.connect('tutorial.db')
c = conn.cursor()

In [9]:
import pymysql


In [10]:
mysql_conn = pymysql.connect(host='demo1007.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                             user='fis_student',
                             password='SuperSecurePassword',
                             db='fis')


In [23]:
mysql_conn = pymysql.connect(host='localhost',
                             user='root',
                             password='password',
                             db='fis')

In [11]:
ms_cur = mysql_conn.cursor()

In [12]:
ms_cur.execute('select * from students limit 2')

2

In [14]:
ms_cur.fetchall()

(('Sean Abu Wilson', '02/06', 2, 'Birmingham, AL', '7.6', 'guacamole'),
 ('David Miller', '06/06', 2, 'New York, NY', '25.75', 'pizza'))

In [2]:
columns = [x[0] for x in c.execute('select * from students').description]

In [3]:
columns

['name',
 'birthdate',
 'siblings',
 'birth_place',
 'years_in_nyc',
 'favorite_food']

In [5]:
query = """
SELECT
  favorite_food, count(*) as number_of_students
FROM students
GROUP BY favorite_food
HAVING COUNT(favorite_food) >= 2;
"""

In [15]:
import pandas as pd
pd.read_sql_query(query, mysql_conn)

,favorite_food,number_of_students
0,pizza,2
1,steak,2


### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

In [25]:
# mysql OR sqlite
pd.read_sql_query("""
SELECT
  name
FROM students""", mysql_conn)

,name
0,Sean Abu Wilson
1,David Miller
2,Abhijeet Kamble
3,Samantha Jackson
4,Anmol Srivats
5,Ran Tokman
6,Amy Li
7,Florencia Leoni
8,Austin Krause
9,Natalie Overchuk


2. Which student has the most siblings?

> This is great place to use a subquery. Encourage students who are initially struggling with a question along the lines of "How could you select the largest number of siblings that anyone has in the group?" From there, you can further push students with a hint if needed: "How can you now make a selection using the result of this, [embedded as a subquery]?"

In [26]:
# mysql OR sqlite
pd.read_sql_query("""
SELECT
  name,
  siblings
FROM students
WHERE siblings = (SELECT MAX(siblings) FROM students);
""", mysql_conn)

,name,siblings
0,Florencia Leoni,4
1,Mohamad Eldebek,4
2,Menachi Korn,4
3,Miguel Peña,4


3. How many students are only children?

In [28]:
# mysql OR sqlite
pd.read_sql_query("""
SELECT
  COUNT(name) AS count
FROM students
WHERE siblings = 0;
""", mysql_conn)

,count
0,3


4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [29]:
# mysql OR sqlite
pd.read_sql_query("""
SELECT
  name,
  years_in_nyc
FROM students
ORDER BY 2
LIMIT 3;
""", mysql_conn)

,name,years_in_nyc
0,Anmol Srivats,0.05
1,Natalie Overchuk,0.10
2,Austin Krause,0.17


5. How many students are native New Yorkers?

In [30]:
# mysql OR sqlite
pd.read_sql_query("""
SELECT
  name,
  birth_place
FROM Students
WHERE LOWER(TRIM(birth_place)) LIKE '%ny';
""", mysql_conn)

,name,birth_place
0,David Miller,"New York, NY"
1,Amy Li,"New York, NY"
2,Akshay Sharma,"New York, NY"
3,Adam Dick,"New York, NY"
4,Alex Mitrani,"New York, NY"
5,Nicole Roach,"Brooklyn, NY"


6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [31]:
# mysql OR sqlite
pd.read_sql_query("""
SELECT
  favorite_food,
  count(favorite_food) AS count
FROM students
GROUP BY favorite_food
HAVING count(favorite_food) > 1;
""", mysql_conn)

,favorite_food,count
0,pizza,2
1,steak,2


In [33]:
# or to get the students that like those foods:
# mysql OR sqlite
pd.read_sql_query("""
SELECT
  name, favorite_food
FROM students
WHERE favorite_food in (SELECT
  favorite_food
FROM students
GROUP BY favorite_food
HAVING count(favorite_food) > 1)
ORDER BY 2;
""", mysql_conn)

,name,favorite_food
0,David Miller,pizza
1,Akshay Sharma,pizza
2,Florencia Leoni,steak
3,Nicole Roach,steak


## More Questions

What are the favorite foods of this classroom?

In [34]:
# mysql OR sqlite
pd.read_sql_query("""
SELECT
  favorite_food,
  COUNT(*) AS count
FROM students 
GROUP BY 1
ORDER BY 2 DESC
""", mysql_conn)

,favorite_food,count
0,pizza,2
1,steak,2
2,Reeses Puffs,1
3,Kare Kare,1
4,guacamole,1
5,pho,1
6,Fusilli Sorrentina,1
7,Biriyani,1
8,burgers,1
9,chocolate chip cookies,1


7. Which student was born closest to the cohort's graduation date?

In [39]:
# mysql
pd.read_sql_query("""
SELECT name,
abs(date(concat('2019-',substr(birthdate,1,2), '-', substr(birthdate,4,2))) - date('2019-06-01')) AS days
FROM students
ORDER BY 2
LIMIT 1
""", mysql_conn)

,name,days
0,David Miller,5


In [37]:
# sqlite
pd.read_sql_query("""
SELECT name,
abs(julianday('2019-' || substr(birthdate,1,2) || '-' || substr(birthdate,4,2)) - julianday('2019-06-01')) AS days
FROM students
ORDER BY 2
LIMIT 1
""", sqlite_conn)

,name,days
0,David Miller,5.0
